<a href="https://colab.research.google.com/github/alirempel/cap-comp215/blob/main/project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2D Circle Simulation
### Author: Alison R
### Date: 2024-02-05

ADD DESCRIPTION OF PROJECT HEREEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE

Below are the imported modules and the constant variables.

In [2]:
import matplotlib.pyplot as plt
import random
import math
import itertools


LEFT = -1
RIGHT = 1
UP = 1
DOWN = -1

###Below are the customizable variables:
* *window_size* is the size you would like the window of the simulation to be (must be a TUPLE containing 2 INTs)
* *num_circles* is the number of circles you would like in the simulation (must be an INT)

  **IMPORTANT NOTE: The below variables are <u>lists</u>, and the length must be the same as num_circles**
* *circle_radii* is a list of the sizes of each circle's radius (each entry must be an INT)
* *circle_positions* is a list of the starting positions for each circle (each entry must be a **TUPLE** with 2 INTs and within the window size)
* *circle_directions* is a list of the circle's directions as angles (each entry must be an INT between 0 and 360)



In [3]:
window_size = (500,500)
x,y = window_size

num_circles = 3

circle_radii = [10,5,20]
circle_positions = [(11,489),(23,100),(300,200)]
circle_directions = [45,360,80]


In [8]:

class Circle:
    """ Representation for the data and operations on a circle """

    def __init__(self, radius:float, centre:tuple, direction:int):
      assert len(centre) == 2
      self.radius = radius
      self.centre = centre
      self.direction = direction
      self.x = round(math.cos(direction),2)
      self.y = round(math.sin(direction),2)

    def __str__(self):
      """ Show the radius and position in a pretty way """
      return f"""Radius: {self.radius}, Position: {self.centre}"""

    def move(self):
        """ Move this circle by given (x,y) offset """
        self.centre = (self.centre[0] + self.x, self.centre[1] + self.y)

    def distance(self, other):
        """ Return the distance between this circle's centre and the other one """
        return math.dist(self.centre, other.centre)

    def intersects(self, other):
        """ Return True iff this circle intersects the other one """
        return self.radius + other.radius > self.distance(other)

In [13]:
# Test Circle class
def test_circle():
  c1 = Circle(3,(2,4),0)
  c2 = Circle(2,(5,8),30)
  assert c1 != c2 and c1.radius != c2.radius and c1.centre != c2.centre

  assert c1.distance(c2) == c2.distance(c1)
  assert c1.distance(c2) == 5.0
  assert c1.intersects(c2) == False
  assert c1.centre == (2,4)
  c1.move()
  assert c1.centre == (3.0,4.0)
  assert c1.intersects(c2) == True


  print('tests passed!!')

test_circle()

tests passed!!


In [14]:

class Simulation:
  """ Object that contains a number of circles that manages methods between the circles """

  def __init__(self,circles:list):
    self.circles = circles

  def move_circles(self):
    """ Loop through circle list and move each one
    Flip the direction (x or y or both) of any circle that is hitting the edge of window """
    for circle in self.circles:

      if (circle.centre[0] + circle.radius) > x and (circle.direction < 90 or circle.direction > 270):
        circle.direction = (circle.direction + 180)%360

      if (circle.centre[0] - circle.radius) < 0 and circle.direction > 90 and circle.direction < 270:
        circle.direction = (circle.direction + 180)%360

      if (circle.centre[1] + circle.radius) > y and circle.direction < 180:
        circle.direction = (circle.direction + 180)%360

      if (circle.centre[1] - circle.radius) < 0 and circle.direction > 180 and circle.direction < 360:
        circle.direction = (circle.direction + 180)%360

      circle.move()


  def check_collision(self):
    """ Loop through each unique pair of circles in circle list and check for collisions
    If there is a collision, the directions of the circles get swapper with each other's """

    for circle_pair in itertools.combinations(self.circles,2):

      if circle_pair[0].intersects(circle_pair[1]):
        circle_pair[0].direction = circle_pair[1].direction
        circle_pair[1].direction = circle_pair[0].direction

  def step(self):
    """ Run the move_circles and check_collisions function """
    self.move_circles()
    self.check_collision()

  def draw(self,axes):
    """ Draw the current state of the simulation """
    axes.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False,
                     labelbottom=False, labeltop=False, labelleft=False, labelright=False, )
    # Defaults for displaying a "matrix" with hard-pixel boundaries and (0,0) at top-left
    options = {**dict(interpolation='nearest', origin='upper'), **options}
    axes.imshow(self.state, cmap=cmap, **options)



circle_list = [(Circle(rad,pos,dir)) for rad,pos,dir in zip(circle_radii,circle_positions,circle_directions)]

a = Simulation(circle_list,(100,200))


TypeError: must be real number, not list

In [ ]:
class Animation2D:
    """
      Animates any 2D model with a step() method and a draw() method, using matplotlib
      model.step() should take no parameters - just step the model forward one step.
      model.draw() should take 2 parameters, the matpltolib axes to draw on and an integer step number

      See https://www.allendowney.com/blog/2019/07/25/matplotlib-animation-in-jupyter/
          for a discussion of the pros and cons of various animation techniques in jupyter notebooks
    """

    def __init__(self, model:Simulation, frames=50, steps_per_frame=1, figsize=(8, 8)):
        """
        :param model: the simulation object to animate, with step() and draw(axes, step) methods
        :param frames: number of animation frames to generate
        """
        self.model = model
        self.frames = frames
        self.steps_per_frame = steps_per_frame
        self.fig, self.ax = plt.subplots(figsize=figsize)

    def animation_step(self, step):
        """ Step the model forward and draw the plot """
        if step > 0:
            for _ in range(self.steps_per_frame):
                self.model.step()
        self.model.draw(self.ax, step=step * self.steps_per_frame)

    def show(self):
        """ return the matplotlib animation object, ready for display """
        anim = animation.FuncAnimation(self.fig, self.animation_step, frames=self.frames)
        plt.close()  # this ensures the last frame is not shown as a separate plot
        return anim

    def animate(self, interval=None):
        """ Animate the model simulation directly in the notebook display block """
        from IPython.display import clear_output
        try:
            for i in range(self.frames):
                clear_output(wait=True)  # clear the IPython display
                self.ax.clear()  # clear old image from the axes (fixes a performance issue)
                plt.figure(self.fig)  # add the figure back to pyplot ** sigh **
                self.animation_step(i)
                plt.show()  # show the current animation frame (pyplot then closes and throws away figure ** sigh **)
                if interval:
                    time.sleep(interval)
        except KeyboardInterrupt:
            pass